### First i will use youtube data Api to get the videos of the BBC Arabic channel
#### To do that i will first get the channel ID then search for 25 playlists in this channel then get the 25 first videos in each playlist

In [1]:
import os

import google.oauth2.credentials

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

# The CLIENT_SECRETS_FILE variable specifies the name of a file that contains
# the OAuth 2.0 information for this application, including its client_id and
# client_secret.
CLIENT_SECRETS_FILE = "client_secret.json"

# This OAuth 2.0 access scope allows for full read/write access to the
# authenticated user's account and requires requests to use an SSL connection.
SCOPES = ['https://www.googleapis.com/auth/youtube']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

def get_authenticated_service():
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
    credentials = flow.run_console()
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def channels_list_by_username(service, **kwargs):
    results = service.channels().list(
        **kwargs
    ).execute()
    print('This channel\'s ID is. Its title is , and it has  views.',
       results['items'][0]['id'])
    return(results['items'][0]['id'])
# Sample python code for playlists.list

def playlists_list_by_channel_id(service, **kwargs):
    response = service.playlists().list(
    **kwargs).execute()
    return response
def playlist_items_list_by_playlist_id(service, **kwargs):
  # See full sample for function
    response = service.playlistItems().list(
    **kwargs).execute()

    return(response)

def videos_list_by_id(service, **kwargs):
    response = service.videos().list(
    **kwargs).execute()
    return response


if __name__ == '__main__':
  # When running locally, disable OAuthlib's HTTPs verification. When
  # running in production *do not* leave this option enabled.
    os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
    service = get_authenticated_service()
    id=channels_list_by_username(service,
          part='snippet,contentDetails,statistics',
          forUsername='BBCArabicNews')
    
    playlists=playlists_list_by_channel_id(service,
        part='snippet,contentDetails',
        channelId=id,
        maxResults=25)
    playlist_ids=[]
    for item in playlists['items']:
        playlist_ids.append(item['id'])
        
    videos_id=[]
    for i in playlist_ids:
        playlist_items=playlist_items_list_by_playlist_id(service,
            part='snippet,contentDetails',
            maxResults=25,
            playlistId=i)
        for item in playlist_items['items']:
            videos_id.append(item['contentDetails']['videoId'])
    data=[]
    for k in videos_id:
        response=videos_list_by_id(service,
            part='snippet,contentDetails,statistics,topicDetails,recordingDetails',
            id=k)
        data.append(response)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=416798608563-re4fh6l9qh90pn0gjrlf9ij7rj5hd4ie.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube&state=whisfJOwaPsFhzYjyrDqGsvDyDqY1j&prompt=consent&access_type=offline
Enter the authorization code: 4/AABhFvFXLVWhZaA1kaU2P6_gnu4QPpUTACbZUVgza_4QWEqRizi7QlM
This channel's ID is. Its title is , and it has  views. UCelk6aHijZq-GJBBB9YpReA


#### Lets look at the response we get from the api

In [2]:
data[1]

{'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/vqtO7_9NSREXLyl9lLWRc9w7d3Y"',
 'items': [{'contentDetails': {'caption': 'false',
    'definition': 'hd',
    'dimension': '2d',
    'duration': 'PT10M3S',
    'licensedContent': True,
    'projection': 'rectangular'},
   'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/JLvat9A71RUyhjgPZEEoFAhessM"',
   'id': '_fdbqhHrbJc',
   'kind': 'youtube#video',
   'snippet': {'categoryId': '25',
    'channelId': 'UCelk6aHijZq-GJBBB9YpReA',
    'channelTitle': 'BBC News عربي',
    'defaultAudioLanguage': 'ar',
    'defaultLanguage': 'ar',
    'description': 'للمزيد من تقارير حسام عاصي السينمائية:\n https://www.youtube.com/playlist?list=PL63lwGZ_8vsloyjQiFZ3eKpzOqgsJ7BRA',
    'liveBroadcastContent': 'none',
    'localized': {'description': 'للمزيد من تقارير حسام عاصي السينمائية:\n https://www.youtube.com/playlist?list=PL63lwGZ_8vsloyjQiFZ3eKpzOqgsJ7BRA',
     'title': 'علي صوفان، عميل الأف بي آي اللبناني الاصل، يتحدث عن مشاركته في الحرب ضد القاعدة'},
    'publishedAt':

In [3]:
len(data)

335

### From the response i will take only the informative attributes like 
#### -Title
#### -PublishedAT
#### -defaultLanguage
#### -duration
#### -Comments count
#### -disLike Counts
#### -Favorite Counts
#### -likes Counts
#### -ViewCounts
#### -Tags

#### i used try exception because i noticed that some vides don't have some attributes 

In [4]:
ls=[]
for item in data:
    dic={}
    try:
        dic['title']=item['items'][0]['snippet']['title'] 
    except:
        dic['title']=None
    try:
        dic['publishedAt']=item['items'][0]['snippet']['publishedAt']
    except:
        dic['publishedAt']=None
    try:
        dic['defaultAudioLanguage']=item['items'][0]['snippet']['defaultAudioLanguage'] 
    except:
        dic['defaultAudioLanguage']=None
    try:
        dic['duration']=item['items'][0]['contentDetails']['duration']
    except:
        dic['duration']=None
    try:
        dic['commentCount']=item['items'][0]['statistics']['commentCount']
    except:
        dic['commentCount']=None
    try:
        dic['dislikeCount']=item['items'][0]['statistics']['dislikeCount']
    except:
        dic['dislikeCount']=None
    try:
        dic['favoriteCount']=item['items'][0]['statistics']['favoriteCount']
    except:
        dic['favoriteCount']=None
    try:
        dic['likeCount']=item['items'][0]['statistics']['likeCount']
    except:
        dic['likeCount']=None
    try:
        dic['viewCount']=item['items'][0]['statistics']['viewCount']
    except:
        dic['viewCount']=None
    try:
        dic['tage']=item['items'][0]['snippet']['tags']
    except:
        dic['tage']=None
    ls.append(dic)

In [5]:
ls

[{'commentCount': '4',
  'defaultAudioLanguage': 'ar',
  'dislikeCount': '3',
  'duration': 'PT10M2S',
  'favoriteCount': '0',
  'likeCount': '27',
  'publishedAt': '2018-07-26T07:42:44.000Z',
  'tage': ['بي بي سي',
   'بي بي سي عربي',
   'بي بي سي العربية',
   'بي بي سي اخبار',
   'اخبار البي بي سي',
   'السيدة مارفيل',
   'كيفين فايغي',
   'المنتقمون',
   'الفهد الاسوب'],
  'title': 'رئيس استوديوهات هارفيل يكشف عن التحضير لصنع فيلم بطلة قوى خارقة',
  'viewCount': '886'},
 {'commentCount': '3',
  'defaultAudioLanguage': 'ar',
  'dislikeCount': '5',
  'duration': 'PT10M3S',
  'favoriteCount': '0',
  'likeCount': '14',
  'publishedAt': '2018-07-23T06:07:41.000Z',
  'tage': ['بي بي سي',
   'بي بي سي عربي',
   'بي بي سي العربية',
   'حسام عاصي',
   'علي صوفان',
   'الابراج المشيدة'],
  'title': 'علي صوفان، عميل الأف بي آي اللبناني الاصل، يتحدث عن مشاركته في الحرب ضد القاعدة',
  'viewCount': '790'},
 {'commentCount': '8',
  'defaultAudioLanguage': 'ar',
  'dislikeCount': '6',
  'duration':

#### Convert to dataframe


In [6]:
import pandas as pd
df=pd.DataFrame(ls)

In [7]:
df

,commentCount,defaultAudioLanguage,dislikeCount,duration,favoriteCount,likeCount,publishedAt,tage,title,viewCount
0,4,ar,3,PT10M2S,0,27,2018-07-26T07:42:44.000Z,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, بي...",رئيس استوديوهات هارفيل يكشف عن التحضير لصنع في...,886
1,3,ar,5,PT10M3S,0,14,2018-07-23T06:07:41.000Z,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, حس...",علي صوفان، عميل الأف بي آي اللبناني الاصل، يتح...,790
2,8,ar,6,PT10M4S,0,44,2018-07-19T12:40:12.000Z,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, بي...",هل حقا أن النجم الاسباني انطونيو بانديراس ينحد...,2580
3,26,ar,5,PT9M59S,0,92,2018-07-16T10:07:10.000Z,"[بي بي سي, بي بي سي عربي, روب راينر, صدمة ورعب...","فيلم ""صدمة ورعب"": لماذا صدّق الاعلام الامريكي ...",3027
4,1,ar,4,PT10M2S,0,23,2018-07-12T09:01:37.000Z,"[بي بي سي عربي, بي بي سي اخبار, حسام عاصي, ستي...",هل عملاء السي آي آى الذين يسربون معلومات سرية ...,760
5,5,ar,5,PT10M3S,0,54,2018-07-09T05:08:55.000Z,"[بي بي سي, بي بي سي عربي, حسام عاصي, هوليوود, ...",هل يمكن لاستوديوهات هوليوود أن تبقى على قيد ال...,2581
6,5,ar,7,PT10M1S,0,40,2018-07-04T05:28:38.000Z,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, بي...",لماذا كانت 2017 الاسوأ لهوليوود والافضل للعرب ...,1972
7,9,ar,4,PT10M3S,0,49,2018-07-02T06:14:14.000Z,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, بي...",من هم السينمائيون العرب الذين تلقوا دعوات للان...,1865
8,11,ar,6,PT10M1S,0,32,2018-06-28T06:25:53.000Z,"[بي بي سي, بي بي سي عربي, اخبار البي بي سي, لو...",كيف تنبأ الكاتب لورنس رايت باحداث سبتمبر ١١ في...,1210
9,1,ar,1,PT10M3S,0,5,2018-06-26T06:31:37.000Z,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, بي...",لماذا فشل سولو-قصة حرب نجوم في شباك التذاكر؟,548


### things to do 
#### 1-delete empty rows
#### 2-fix empty coulums
#### 3-fix the published date col
#### 4-fix the duration col

In [8]:
import numpy as np
df = df.replace(to_replace='None', value=np.nan).dropna()

In [9]:
df

,commentCount,defaultAudioLanguage,dislikeCount,duration,favoriteCount,likeCount,publishedAt,tage,title,viewCount
0,4,ar,3,PT10M2S,0,27,2018-07-26T07:42:44.000Z,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, بي...",رئيس استوديوهات هارفيل يكشف عن التحضير لصنع في...,886
1,3,ar,5,PT10M3S,0,14,2018-07-23T06:07:41.000Z,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, حس...",علي صوفان، عميل الأف بي آي اللبناني الاصل، يتح...,790
2,8,ar,6,PT10M4S,0,44,2018-07-19T12:40:12.000Z,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, بي...",هل حقا أن النجم الاسباني انطونيو بانديراس ينحد...,2580
3,26,ar,5,PT9M59S,0,92,2018-07-16T10:07:10.000Z,"[بي بي سي, بي بي سي عربي, روب راينر, صدمة ورعب...","فيلم ""صدمة ورعب"": لماذا صدّق الاعلام الامريكي ...",3027
4,1,ar,4,PT10M2S,0,23,2018-07-12T09:01:37.000Z,"[بي بي سي عربي, بي بي سي اخبار, حسام عاصي, ستي...",هل عملاء السي آي آى الذين يسربون معلومات سرية ...,760
5,5,ar,5,PT10M3S,0,54,2018-07-09T05:08:55.000Z,"[بي بي سي, بي بي سي عربي, حسام عاصي, هوليوود, ...",هل يمكن لاستوديوهات هوليوود أن تبقى على قيد ال...,2581
6,5,ar,7,PT10M1S,0,40,2018-07-04T05:28:38.000Z,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, بي...",لماذا كانت 2017 الاسوأ لهوليوود والافضل للعرب ...,1972
7,9,ar,4,PT10M3S,0,49,2018-07-02T06:14:14.000Z,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, بي...",من هم السينمائيون العرب الذين تلقوا دعوات للان...,1865
8,11,ar,6,PT10M1S,0,32,2018-06-28T06:25:53.000Z,"[بي بي سي, بي بي سي عربي, اخبار البي بي سي, لو...",كيف تنبأ الكاتب لورنس رايت باحداث سبتمبر ١١ في...,1210
9,1,ar,1,PT10M3S,0,5,2018-06-26T06:31:37.000Z,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, بي...",لماذا فشل سولو-قصة حرب نجوم في شباك التذاكر؟,548


In [10]:
import dateutil.parser
df['date']=df['publishedAt'].apply(lambda x:dateutil.parser.parse(x).date())
df['time']=df['publishedAt'].apply(lambda x:dateutil.parser.parse(x).time())

In [11]:
df

,commentCount,defaultAudioLanguage,dislikeCount,duration,favoriteCount,likeCount,publishedAt,tage,title,viewCount,date,time
0,4,ar,3,PT10M2S,0,27,2018-07-26T07:42:44.000Z,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, بي...",رئيس استوديوهات هارفيل يكشف عن التحضير لصنع في...,886,2018-07-26,07:42:44
1,3,ar,5,PT10M3S,0,14,2018-07-23T06:07:41.000Z,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, حس...",علي صوفان، عميل الأف بي آي اللبناني الاصل، يتح...,790,2018-07-23,06:07:41
2,8,ar,6,PT10M4S,0,44,2018-07-19T12:40:12.000Z,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, بي...",هل حقا أن النجم الاسباني انطونيو بانديراس ينحد...,2580,2018-07-19,12:40:12
3,26,ar,5,PT9M59S,0,92,2018-07-16T10:07:10.000Z,"[بي بي سي, بي بي سي عربي, روب راينر, صدمة ورعب...","فيلم ""صدمة ورعب"": لماذا صدّق الاعلام الامريكي ...",3027,2018-07-16,10:07:10
4,1,ar,4,PT10M2S,0,23,2018-07-12T09:01:37.000Z,"[بي بي سي عربي, بي بي سي اخبار, حسام عاصي, ستي...",هل عملاء السي آي آى الذين يسربون معلومات سرية ...,760,2018-07-12,09:01:37
5,5,ar,5,PT10M3S,0,54,2018-07-09T05:08:55.000Z,"[بي بي سي, بي بي سي عربي, حسام عاصي, هوليوود, ...",هل يمكن لاستوديوهات هوليوود أن تبقى على قيد ال...,2581,2018-07-09,05:08:55
6,5,ar,7,PT10M1S,0,40,2018-07-04T05:28:38.000Z,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, بي...",لماذا كانت 2017 الاسوأ لهوليوود والافضل للعرب ...,1972,2018-07-04,05:28:38
7,9,ar,4,PT10M3S,0,49,2018-07-02T06:14:14.000Z,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, بي...",من هم السينمائيون العرب الذين تلقوا دعوات للان...,1865,2018-07-02,06:14:14
8,11,ar,6,PT10M1S,0,32,2018-06-28T06:25:53.000Z,"[بي بي سي, بي بي سي عربي, اخبار البي بي سي, لو...",كيف تنبأ الكاتب لورنس رايت باحداث سبتمبر ١١ في...,1210,2018-06-28,06:25:53
9,1,ar,1,PT10M3S,0,5,2018-06-26T06:31:37.000Z,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, بي...",لماذا فشل سولو-قصة حرب نجوم في شباك التذاكر؟,548,2018-06-26,06:31:37


In [13]:
def clean_time(text):
    ls=re.findall(r"\d+", text)
    if len(ls)>1:
        return str(ls[0]+'M')
    else:
        return "<1M"

In [15]:
import re
df['duration']=df['duration'].apply(lambda x: clean_time(x))

In [16]:
del df['publishedAt']

In [17]:
df

,commentCount,defaultAudioLanguage,dislikeCount,duration,favoriteCount,likeCount,tage,title,viewCount,date,time
0,4,ar,3,10M,0,27,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, بي...",رئيس استوديوهات هارفيل يكشف عن التحضير لصنع في...,886,2018-07-26,07:42:44
1,3,ar,5,10M,0,14,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, حس...",علي صوفان، عميل الأف بي آي اللبناني الاصل، يتح...,790,2018-07-23,06:07:41
2,8,ar,6,10M,0,44,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, بي...",هل حقا أن النجم الاسباني انطونيو بانديراس ينحد...,2580,2018-07-19,12:40:12
3,26,ar,5,9M,0,92,"[بي بي سي, بي بي سي عربي, روب راينر, صدمة ورعب...","فيلم ""صدمة ورعب"": لماذا صدّق الاعلام الامريكي ...",3027,2018-07-16,10:07:10
4,1,ar,4,10M,0,23,"[بي بي سي عربي, بي بي سي اخبار, حسام عاصي, ستي...",هل عملاء السي آي آى الذين يسربون معلومات سرية ...,760,2018-07-12,09:01:37
5,5,ar,5,10M,0,54,"[بي بي سي, بي بي سي عربي, حسام عاصي, هوليوود, ...",هل يمكن لاستوديوهات هوليوود أن تبقى على قيد ال...,2581,2018-07-09,05:08:55
6,5,ar,7,10M,0,40,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, بي...",لماذا كانت 2017 الاسوأ لهوليوود والافضل للعرب ...,1972,2018-07-04,05:28:38
7,9,ar,4,10M,0,49,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, بي...",من هم السينمائيون العرب الذين تلقوا دعوات للان...,1865,2018-07-02,06:14:14
8,11,ar,6,10M,0,32,"[بي بي سي, بي بي سي عربي, اخبار البي بي سي, لو...",كيف تنبأ الكاتب لورنس رايت باحداث سبتمبر ١١ في...,1210,2018-06-28,06:25:53
9,1,ar,1,10M,0,5,"[بي بي سي, بي بي سي عربي, بي بي سي العربية, بي...",لماذا فشل سولو-قصة حرب نجوم في شباك التذاكر؟,548,2018-06-26,06:31:37


In [18]:
df.to_csv("clean_data")